In [302]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [156]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, GlobalMaxPooling1D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
try:
    tf.set_random_seed(1337)                    # set the random seed for reproducibility
except:
    tf.random.set_seed(1337)                     # NOTE: Newer version of tensorflow uses tf.random.set_seed
np.random.seed(1337)                         #       in

In [602]:
#Preparación datos 
Ventas=pd.read_csv('./Input2_clientes_venta.csv', sep=';')
Clientes=pd.read_csv('./Input1_clientes_estructura.csv', sep=';')
Ventas["Marcas"]=Ventas["Marca2"]+'-'+Ventas["Cupo2"]+'-'+Ventas["CapacidadEnvase2"]
Ventas["Periodo"]=Ventas["Año"].astype('str')+"-"+Ventas["Mes"].astype('str')
Ventas["Descuento"]=Ventas.disc.apply(lambda x: 1 if x!=0 else 0)
Ventas["Obsequio"]=Ventas.nr.apply(lambda x: 0 if x!=0 else 1)
Clientes['Gerencia2']=Clientes['Gerencia2'].str.replace('Gerencia_','')
Clientes['SubCanal2']=Clientes['SubCanal2'].str.replace('Subcanal_','')
Clientes['Categoria']=Clientes['Categoria'].str.replace('Categoria_','')
Clientes=Clientes.astype({"Gerencia2":'int',"SubCanal2":'int',"Categoria":'int'}) 
Clientes=Clientes.drop(columns=['Regional2'])
Training=pd.read_csv('./Input3_clientes_test.csv', sep=';')
Base_T=Training["Cliente"]
Training=pd.read_csv('./Input3_clientes_test.csv', sep=';')
Base_T=pd.DataFrame(Training["Cliente"])
Base_T
Ventas_20208=Ventas[Ventas["Periodo"]=="2020-8"]
Ventas_20208["CupoP"]=Ventas_20208.Cupo2.apply(lambda x: 1 if ((x=="Cupo_2") or (x=="Cupo_3")) else 0)
Ventas_20208=Ventas_20208.drop(columns=['Cupo2'])
Ventas_20208=Ventas_20208.drop(columns=['SegmentoPrecio2'])
Ventas_20208=Ventas_20208.drop(columns=['Año','Mes',])
Ventas_20208["Marca_20"]=Ventas_20208.Marca2.apply(lambda x: 1 if (x=="Marca_20") else 0)
Ventas_20208["Marca_16"]=Ventas_20208.Marca2.apply(lambda x: 1 if (x=="Marca_16") else 0)
Ventas_20208["Marca_9"]=Ventas_20208.Marca2.apply(lambda x: 1 if (x=="Marca_9") else 0)
Ventas_20208["Marca_38"]=Ventas_20208.Marca2.apply(lambda x: 1 if (x=="Marca_38") else 0)
Ventas_20208["Marca_39"]=Ventas_20208.Marca2.apply(lambda x: 1 if (x=="Marca_39") else 0)
Ventas_20208["CapacidadEnvase_9"]=Ventas_20208.CapacidadEnvase2.apply(lambda x: 1 if (x=="CapacidadEnvase_9") else 0)
Ventas_20208["CapacidadEnvase_10"]=Ventas_20208.CapacidadEnvase2.apply(lambda x: 1 if (x=="CapacidadEnvase_10") else 0)
Ventas_20208["CapacidadEnvase_12"]=Ventas_20208.CapacidadEnvase2.apply(lambda x: 1 if (x=="CapacidadEnvase_12") else 0)
Ventas_20208["Ventas"]=Ventas_20208.Marcas.apply(lambda x: x if ((x=="Marca_20-Cupo_3-CapacidadEnvase_9") 
                                                                 or (x=="Marca_16-Cupo_2-CapacidadEnvase_10")
                                                                or (x=="Marca_9-Cupo_3-CapacidadEnvase_12")
                                                                or (x=="Marca_38-Cupo_2-CapacidadEnvase_10")
                                                                or (x=="Marca_39-Cupo_2-CapacidadEnvase_10")) else "Otra")
Ventas_20208=Ventas_20208.drop(columns=['Marca2','disc','CapacidadEnvase2'])
Ventas_20208=Ventas_20208.replace("Marca_20-Cupo_3-CapacidadEnvase_9","Marca1")
Ventas_20208=Ventas_20208.replace("Marca_16-Cupo_2-CapacidadEnvase_10","Marca2")
Ventas_20208=Ventas_20208.replace("Marca_9-Cupo_3-CapacidadEnvase_12","Marca3")
Ventas_20208=Ventas_20208.replace("Marca_38-Cupo_2-CapacidadEnvase_10","Marca4")
Ventas_20208=Ventas_20208.replace("Marca_39-Cupo_2-CapacidadEnvase_10","Marca5")
Ventas_20208=Ventas_20208.drop(columns=['Marcas','Periodo'])
volumen=pd.pivot_table(Ventas_20208, values=['Volumen','nr'], index=['Cliente'], columns=['Ventas'], aggfunc=np.sum, fill_value=0)
volumen.columns = volumen.columns.droplevel(0)
volumen = volumen.reset_index().rename_axis(None, axis=1)
Ventas=pd.DataFrame(Ventas_20208.groupby(by=["Cliente"]).sum()).reset_index()
Datapreparada= pd.merge(volumen, Ventas, how='left',  on='Cliente')
Datapreparada

C:\Users\camil\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Cliente,Marca1,Marca2,Marca3,Marca4,Marca5,Otra,Marca1,Marca2,Marca3,...,Obsequio,CupoP,Marca_20,Marca_16,Marca_9,Marca_38,Marca_39,CapacidadEnvase_9,CapacidadEnvase_10,CapacidadEnvase_12
0,1,0.000000,0.0,0.0,0.078244,0.000000,0.130406,0.000000,0.0,0.0,...,1,3,0,0,0,1,1,0,3,0
1,2,0.000000,0.0,0.0,0.117366,0.000000,0.888634,0.000000,0.0,0.0,...,1,4,1,0,0,1,1,0,4,1
2,3,0.000000,0.0,0.0,0.000000,0.000000,0.086938,0.000000,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
3,4,0.101164,0.0,0.0,0.136927,0.000000,0.918509,632.820727,0.0,0.0,...,1,6,2,0,0,1,1,1,5,0
4,5,0.050582,0.0,0.0,0.000000,0.000000,3.578361,316.410336,0.0,0.0,...,0,5,1,0,1,0,0,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13261,20414,0.000000,0.0,0.0,0.019561,0.000000,0.000000,0.000000,0.0,0.0,...,1,2,0,0,0,1,1,0,2,0
13262,20462,0.000000,0.0,0.0,0.097805,0.039122,0.000000,0.000000,0.0,0.0,...,0,2,0,0,0,1,1,0,2,0
13263,20529,0.000000,0.0,0.0,0.880243,0.000000,0.000000,0.000000,0.0,0.0,...,1,2,0,0,0,1,1,0,2,0
13264,20567,0.000000,0.0,0.0,0.019561,0.019561,0.000000,0.000000,0.0,0.0,...,0,2,0,0,0,1,1,0,2,0


In [603]:
Periodo1=Datapreparada
Periodo1

,Cliente,Marca1,Marca2,Marca3,Marca4,Marca5,Otra,Marca1,Marca2,Marca3,...,Obsequio,CupoP,Marca_20,Marca_16,Marca_9,Marca_38,Marca_39,CapacidadEnvase_9,CapacidadEnvase_10,CapacidadEnvase_12
0,1,0.000000,0.0,0.0,0.078244,0.000000,0.130406,0.000000,0.0,0.0,...,1,3,0,0,0,1,1,0,3,0
1,2,0.000000,0.0,0.0,0.117366,0.000000,0.888634,0.000000,0.0,0.0,...,1,4,1,0,0,1,1,0,4,1
2,3,0.000000,0.0,0.0,0.000000,0.000000,0.086938,0.000000,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
3,4,0.101164,0.0,0.0,0.136927,0.000000,0.918509,632.820727,0.0,0.0,...,1,6,2,0,0,1,1,1,5,0
4,5,0.050582,0.0,0.0,0.000000,0.000000,3.578361,316.410336,0.0,0.0,...,0,5,1,0,1,0,0,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13261,20414,0.000000,0.0,0.0,0.019561,0.000000,0.000000,0.000000,0.0,0.0,...,1,2,0,0,0,1,1,0,2,0
13262,20462,0.000000,0.0,0.0,0.097805,0.039122,0.000000,0.000000,0.0,0.0,...,0,2,0,0,0,1,1,0,2,0
13263,20529,0.000000,0.0,0.0,0.880243,0.000000,0.000000,0.000000,0.0,0.0,...,1,2,0,0,0,1,1,0,2,0
13264,20567,0.000000,0.0,0.0,0.019561,0.019561,0.000000,0.000000,0.0,0.0,...,0,2,0,0,0,1,1,0,2,0


In [607]:
#Preparación datos 
Ventas=pd.read_csv('./Input2_clientes_venta.csv', sep=';')
Clientes=pd.read_csv('./Input1_clientes_estructura.csv', sep=';')
Ventas["Marcas"]=Ventas["Marca2"]+'-'+Ventas["Cupo2"]+'-'+Ventas["CapacidadEnvase2"]
Ventas["Periodo"]=Ventas["Año"].astype('str')+"-"+Ventas["Mes"].astype('str')
Ventas["Descuento"]=Ventas.disc.apply(lambda x: 1 if x!=0 else 0)
Ventas["Obsequio"]=Ventas.nr.apply(lambda x: 0 if x!=0 else 1)
Clientes['Gerencia2']=Clientes['Gerencia2'].str.replace('Gerencia_','')
Clientes['SubCanal2']=Clientes['SubCanal2'].str.replace('Subcanal_','')
Clientes['Categoria']=Clientes['Categoria'].str.replace('Categoria_','')
Clientes=Clientes.astype({"Gerencia2":'int',"SubCanal2":'int',"Categoria":'int'}) 
Clientes=Clientes.drop(columns=['Regional2'])
Training=pd.read_csv('./Input3_clientes_test.csv', sep=';')
Base_T=Training["Cliente"]
Training=pd.read_csv('./Input3_clientes_test.csv', sep=';')
Base_T=pd.DataFrame(Training["Cliente"])
Base_T
Ventas_20209=Ventas[Ventas["Periodo"]=="2020-9"]
Ventas_20209["CupoP"]=Ventas_20209.Cupo2.apply(lambda x: 1 if ((x=="Cupo_2") or (x=="Cupo_3")) else 0)
Ventas_20209=Ventas_20209.drop(columns=['Cupo2'])
Ventas_20209=Ventas_20209.drop(columns=['SegmentoPrecio2'])
Ventas_20209=Ventas_20209.drop(columns=['Año','Mes',])
Ventas_20209["Marca_20"]=Ventas_20209.Marca2.apply(lambda x: 1 if (x=="Marca_20") else 0)
Ventas_20209["Marca_16"]=Ventas_20209.Marca2.apply(lambda x: 1 if (x=="Marca_16") else 0)
Ventas_20209["Marca_9"]=Ventas_20209.Marca2.apply(lambda x: 1 if (x=="Marca_9") else 0)
Ventas_20209["Marca_38"]=Ventas_20209.Marca2.apply(lambda x: 1 if (x=="Marca_38") else 0)
Ventas_20209["Marca_39"]=Ventas_20209.Marca2.apply(lambda x: 1 if (x=="Marca_39") else 0)
Ventas_20209["CapacidadEnvase_9"]=Ventas_20209.CapacidadEnvase2.apply(lambda x: 1 if (x=="CapacidadEnvase_9") else 0)
Ventas_20209["CapacidadEnvase_10"]=Ventas_20209.CapacidadEnvase2.apply(lambda x: 1 if (x=="CapacidadEnvase_10") else 0)
Ventas_20209["CapacidadEnvase_12"]=Ventas_20209.CapacidadEnvase2.apply(lambda x: 1 if (x=="CapacidadEnvase_12") else 0)
Ventas_20209["Ventas"]=Ventas_20209.Marcas.apply(lambda x: x if ((x=="Marca_20-Cupo_3-CapacidadEnvase_9") 
                                                                 or (x=="Marca_16-Cupo_2-CapacidadEnvase_10")
                                                                or (x=="Marca_9-Cupo_3-CapacidadEnvase_12")
                                                                or (x=="Marca_38-Cupo_2-CapacidadEnvase_10")
                                                                or (x=="Marca_39-Cupo_2-CapacidadEnvase_10")) else "Otra")
Ventas_20209=Ventas_20209.drop(columns=['Marca2','disc','CapacidadEnvase2'])
Ventas_20209=Ventas_20209.replace("Marca_20-Cupo_3-CapacidadEnvase_9","Marca1")
Ventas_20209=Ventas_20209.replace("Marca_16-Cupo_2-CapacidadEnvase_10","Marca2")
Ventas_20209=Ventas_20209.replace("Marca_9-Cupo_3-CapacidadEnvase_12","Marca3")
Ventas_20209=Ventas_20209.replace("Marca_38-Cupo_2-CapacidadEnvase_10","Marca4")
Ventas_20209=Ventas_20209.replace("Marca_39-Cupo_2-CapacidadEnvase_10","Marca5")
Ventas_20209=Ventas_20209.drop(columns=['Marcas','Periodo'])
volumen=pd.pivot_table(Ventas_20209, values=['Volumen','nr'], index=['Cliente'], columns=['Ventas'], aggfunc=np.sum, fill_value=0)
volumen.columns = volumen.columns.droplevel(0)
volumen = volumen.reset_index().rename_axis(None, axis=1)
Ventas=pd.DataFrame(Ventas_20209.groupby(by=["Cliente"]).sum()).reset_index()
Datapreparada= pd.merge(volumen, Ventas, how='left',  on='Cliente')
Periodo2=Datapreparada
Periodo2


Ventas=Periodo2.iloc[:,0:6]
Ventas["M1"]=Ventas["Marca1"].apply(lambda x: 1 if (x>0) else 0)
Ventas["M2"]=Ventas["Marca2"].apply(lambda x: 1 if (x>0) else 0)
Ventas["M3"]=Ventas["Marca3"].apply(lambda x: 1 if (x>0) else 0)
Ventas["M4"]=Ventas["Marca4"].apply(lambda x: 1 if (x>0) else 0)
Ventas["M5"]=Ventas["Marca5"].apply(lambda x: 1 if (x>0) else 0)
Ventas=Ventas.drop(columns=['Marca1','Marca2','Marca3','Marca4','Marca5'])
Ventas

Ventas_Training=pd.merge(Periodo2["Cliente"], Ventas, how='left',  on='Cliente')
Ventas_Training=Ventas_Training.fillna(0)
Ventas_Training

C:\Users\camil\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Cliente,M1,M2,M3,M4,M5
0,1,0,0,0,1,0
1,2,1,0,0,0,0
2,3,0,0,0,1,0
3,4,0,0,0,0,0
4,5,0,0,0,0,0
...,...,...,...,...,...,...
13659,20411,0,0,0,1,1
13660,20414,0,0,0,1,0
13661,20434,0,0,0,1,0
13662,20456,0,0,0,1,1


In [159]:
Periodo1.iloc[:,1:26]

,Marca1,Marca2,Marca3,Marca4,Marca5,Otra,Marca1,Marca2,Marca3,Marca4,...,Obsequio,CupoP,Marca_20,Marca_16,Marca_9,Marca_38,Marca_39,CapacidadEnvase_9,CapacidadEnvase_10,CapacidadEnvase_12
0,0.000000,0.0,0.0,0.078244,0.000000,0.130406,0.000000,0.0,0.0,488.267995,...,1,3,0,0,0,1,1,0,3,0
1,0.000000,0.0,0.0,0.117366,0.000000,0.888634,0.000000,0.0,0.0,719.720948,...,1,4,1,0,0,1,1,0,4,1
2,0.000000,0.0,0.0,0.000000,0.000000,0.086938,0.000000,0.0,0.0,0.000000,...,0,1,0,0,0,0,0,0,1,0
3,0.101164,0.0,0.0,0.136927,0.000000,0.918509,632.820727,0.0,0.0,815.028471,...,1,6,2,0,0,1,1,1,5,0
4,0.050582,0.0,0.0,0.000000,0.000000,3.578361,316.410336,0.0,0.0,0.000000,...,0,5,1,0,1,0,0,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13261,0.000000,0.0,0.0,0.019561,0.000000,0.000000,0.000000,0.0,0.0,114.742688,...,1,2,0,0,0,1,1,0,2,0
13262,0.000000,0.0,0.0,0.097805,0.039122,0.000000,0.000000,0.0,0.0,593.298287,...,0,2,0,0,0,1,1,0,2,0
13263,0.000000,0.0,0.0,0.880243,0.000000,0.000000,0.000000,0.0,0.0,5408.754393,...,1,2,0,0,0,1,1,0,2,0
13264,0.000000,0.0,0.0,0.019561,0.019561,0.000000,0.000000,0.0,0.0,121.146074,...,0,2,0,0,0,1,1,0,2,0


In [380]:
Ventas1=pd.read_csv('./Input2_clientes_venta.csv', sep=';')

In [396]:
Ventas1=Ventas1[Ventas1["Año"]==2020]
VentasR=Ventas1.groupby(by=["Cliente"]).size().reset_index(name='Conteo')
VentasR

,Cliente,Conteo
0,1,12
1,2,32
2,3,11
3,4,47
4,5,65
...,...,...
16342,20529,4
16343,20567,2
16344,20569,2
16345,20577,2


In [412]:
Ventas2=pd.read_csv('./Input2_clientes_venta.csv', sep=';')
Ventas2
Ventas2=Ventas2[(Ventas2["Año"]==2020) & (Ventas2["Mes"]>=6) &   (Ventas2["Mes"]<=8)]
VentasS=Ventas2.groupby(by=["Cliente"]).size().reset_index(name='Conteo1')
VentasS


Ventas2=pd.read_csv('./Input2_clientes_venta.csv', sep=';')
Ventas2
Ventas2=Ventas2[(Ventas2["Año"]==2020) & (Ventas2["Mes"]>=7) &   (Ventas2["Mes"]<=9)]
VentasT=Ventas2.groupby(by=["Cliente"]).size().reset_index(name='Conteo1')
VentasT

,Cliente,Conteo1
0,1,7
1,2,16
2,3,4
3,4,18
4,5,16
...,...,...
15256,20529,4
15257,20567,2
15258,20569,2
15259,20577,2


In [581]:
Ventas3=pd.read_csv('./Input2_clientes_venta.csv', sep=';')
Ventas3
Ventas3=Ventas3[(Ventas3["Año"]==2020) & (Ventas3["Mes"]>=7) &   (Ventas3["Mes"]<=8)& (Ventas3["Marca2"]!="Marca_1") &   (Ventas3["Marca2"]!="Marca_4")]
Ventas3=pd.pivot_table(Ventas3, values=['Volumen'], index=['Cliente'], columns=['Mes'], aggfunc=np.sum, fill_value=0)
Ventas3.columns = Ventas3.columns.droplevel(0)
Ventas3 = Ventas3.reset_index().rename_axis(None, axis=1)
Ventas3.columns = ['Cliente','mes1',"mes2"]
Ventas3["Conditional"]=np.where(Ventas3['mes2']<Ventas3['mes1'], 1, 2)
Ventas3=Ventas3[["Cliente","Conditional"]]
Ventas3


,Cliente,Conditional
0,1,2
1,2,2
2,4,1
3,5,1
4,6,1
...,...,...
13237,20529,2
13238,20567,2
13239,20569,1
13240,20577,2


In [651]:
Ventas5=pd.read_csv('./Input2_clientes_venta.csv', sep=';')
Ventas5
Ventas5=Ventas5[(Ventas5["Año"]==2020) & (Ventas5["Mes"]>=7) &   (Ventas5["Mes"]<=8)& ((Ventas5["Marca2"]=="Marca_1") |   (Ventas5["Marca2"]=="Marca_4"))]
Ventas5=pd.pivot_table(Ventas5, values=['Volumen'], index=['Cliente'], columns=['Mes'], aggfunc=np.sum, fill_value=0)
Ventas5.columns = Ventas5.columns.droplevel(0)
Ventas5 = Ventas5.reset_index().rename_axis(None, axis=1)
Ventas5.columns = ['Cliente','mes1',"mes2"]
Ventas5["Conditional1"]=np.where(Ventas5['mes2']<Ventas5['mes1'], 1, 2)
Ventas5=Ventas5[["Cliente","Conditional1"]]
Ventas5

,Cliente,Conditional1
0,1,2
1,2,2
2,3,2
3,4,2
4,5,2
...,...,...
12576,19284,2
12577,19291,2
12578,19298,2
12579,19307,2


In [679]:
Ventas4=pd.read_csv('./Input2_clientes_venta.csv', sep=';')
Ventas4
Ventas4=Ventas4[(Ventas4["Año"]==2020) & (Ventas4["Mes"]>=8) &   (Ventas4["Mes"]<=9)& (Ventas4["Marca2"]!="Marca_1") &   (Ventas4["Marca2"]!="Marca_4")]
Ventas4=pd.pivot_table(Ventas4, values=['Volumen'], index=['Cliente'], columns=['Mes'], aggfunc=np.sum, fill_value=0)
Ventas4.columns = Ventas4.columns.droplevel(0)
Ventas4 = Ventas4.reset_index().rename_axis(None, axis=1)
Ventas4.columns = ['Cliente','mes1',"mes2"]
Ventas4["Conditional"]=np.where(Ventas4['mes2']<Ventas4['mes1'], 1, 2)
Ventas4=Ventas4[["Cliente","Conditional"]]
Ventas4

,Cliente,Conditional
0,1,2
1,2,1
2,3,2
3,4,1
4,5,1
...,...,...
13918,20462,1
13919,20529,1
13920,20567,1
13921,20577,1


In [680]:
Ventas6=pd.read_csv('./Input2_clientes_venta.csv', sep=';')
Ventas6
Ventas6=Ventas6[(Ventas6["Año"]==2020) & (Ventas6["Mes"]>=8) &   (Ventas6["Mes"]<=9)& ((Ventas6["Marca2"]=="Marca_1") |   (Ventas6["Marca2"]=="Marca_4"))]
Ventas6=pd.pivot_table(Ventas6, values=['Volumen'], index=['Cliente'], columns=['Mes'], aggfunc=np.sum, fill_value=0)
Ventas6.columns = Ventas6.columns.droplevel(0)
Ventas6 = Ventas6.reset_index().rename_axis(None, axis=1)
Ventas6.columns = ['Cliente','mes1',"mes2"]
Ventas6["Conditional1"]=np.where(Ventas6['mes2']<Ventas6['mes1'], 1, 2)
Ventas6=Ventas6[["Cliente","Conditional1"]]
Ventas6

,Cliente,Conditional1
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
13228,19871,2
13229,19876,2
13230,19885,2
13231,19891,2


In [686]:
Ventas7.head()

,Año,Mes,Cliente,SegmentoPrecio2,Marca2,Cupo2,CapacidadEnvase2,Volumen,disc,nr
0,2019,5,10,SegmentoPrecio_1,Marca_1,Cupo_1,CapacidadEnvase_12,0.112229,-30.590603,900.328567
1,2019,5,10,SegmentoPrecio_1,Marca_2,Cupo_2,CapacidadEnvase_10,0.021734,0.000000,149.184463
2,2019,5,10,SegmentoPrecio_2,Marca_3,Cupo_2,CapacidadEnvase_10,0.043469,0.000000,359.625828
3,2019,5,10,SegmentoPrecio_1,Marca_1,Cupo_2,CapacidadEnvase_16,0.026345,-31.065261,134.748399
4,2019,5,10,SegmentoPrecio_1,Marca_4,Cupo_2,CapacidadEnvase_10,0.086938,0.000000,496.901005


In [825]:
Ventas8=pd.read_csv('./Input2_clientes_venta.csv', sep=';')
Ventas8=Ventas8[(Ventas8["Año"]==2020) & (Ventas8["Mes"]>=8) &   (Ventas8["Mes"]<=9)& (Ventas8["Marca2"]!="Marca_1") &   (Ventas8["Marca2"]!="Marca_4")]
Ventas8=pd.pivot_table(Ventas8, values=['nr'], index=['Cliente'], columns=['Mes'], aggfunc=np.sum, fill_value=0)
Ventas8.columns = Ventas8.columns.droplevel(0)
Ventas8 = Ventas8.reset_index().rename_axis(None, axis=1)
Ventas8.columns = ['Cliente','mes1',"mes2"]
Ventas8["Conditional2"]=np.where(Ventas8['mes2']<Ventas8['mes1'], 1, 2)
Ventas8=Ventas8[["Cliente","Conditional2"]]
Ventas8

,Cliente,Conditional2
0,1,1
1,2,1
2,3,2
3,4,1
4,5,1
...,...,...
13918,20462,1
13919,20529,1
13920,20567,1
13921,20577,1


In [430]:
Ventas2=pd.read_csv('./Input2_clientes_venta.csv', sep=';')
Ventas2



,Año,Mes,Cliente,SegmentoPrecio2,Marca2,Cupo2,CapacidadEnvase2,Volumen,disc,nr
0,2019,5,10,SegmentoPrecio_1,Marca_1,Cupo_1,CapacidadEnvase_12,0.112229,-30.590603,900.328567
1,2019,5,10,SegmentoPrecio_1,Marca_2,Cupo_2,CapacidadEnvase_10,0.021734,0.000000,149.184463
2,2019,5,10,SegmentoPrecio_2,Marca_3,Cupo_2,CapacidadEnvase_10,0.043469,0.000000,359.625828
3,2019,5,10,SegmentoPrecio_1,Marca_1,Cupo_2,CapacidadEnvase_16,0.026345,-31.065261,134.748399
4,2019,5,10,SegmentoPrecio_1,Marca_4,Cupo_2,CapacidadEnvase_10,0.086938,0.000000,496.901005
...,...,...,...,...,...,...,...,...,...,...
1406111,2020,8,20577,SegmentoPrecio_3,Marca_39,Cupo_2,CapacidadEnvase_10,0.039122,0.000000,544.150314
1406112,2020,7,20580,SegmentoPrecio_1,Marca_38,Cupo_2,CapacidadEnvase_10,0.000000,0.000000,0.000000
1406113,2020,7,20580,SegmentoPrecio_3,Marca_39,Cupo_2,CapacidadEnvase_10,0.058683,0.000000,759.218996
1406114,2020,9,20580,SegmentoPrecio_1,Marca_38,Cupo_2,CapacidadEnvase_10,0.000000,0.000000,0.000000


In [158]:
Ventas_Training

,Cliente,M1,M2,M3,M4,M5
0,1,0,0,0,1,0
1,2,1,0,0,0,0
2,3,0,0,0,1,0
3,4,0,0,0,0,0
4,5,0,0,0,0,0
...,...,...,...,...,...,...
13659,20411,0,0,0,1,1
13660,20414,0,0,0,1,0
13661,20434,0,0,0,1,0
13662,20456,0,0,0,1,1


In [165]:
import matplotlib.pyplot as plt
import numpy  as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as sfm

from matplotlib.widgets import Slider, Button, RadioButtons
from scipy import interp
from scipy.optimize import fsolve
from scipy.stats import chi2_contingency, ttest_ind
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from statsmodels.formula.api import ols


In [878]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(Datoslogit2.M1, 1/(1+np.exp(-lm2.predict(Datoslogit2)))))
print(roc_auc_score(Datoslogit2.M2, 1/(1+np.exp(-lm3.predict(Datoslogit2)))))
print(roc_auc_score(Datoslogit2.M3, 1/(1+np.exp(-lm4.predict(Datoslogit2)))))
print(roc_auc_score(Datoslogit2.M4, 1/(1+np.exp(-lm5.predict(Datoslogit2)))))
print(roc_auc_score(Datoslogit2.M5, 1/(1+np.exp(-lm6.predict(Datoslogit2)))))

0.8373244598461604
0.8376941208370279
0.85178224886899
0.843078875783646
0.8578406061985393


In [463]:
Datoslogit2= pd.merge(Datoslogit, Clientes, how='left',  on='Cliente')

In [466]:
Datoslogit2
Datoslogit2=Datoslogit2.fillna(0)
Datoslogit2

,Cliente,Marca1,Marca2,Marca3,Marca4,Marca5,Otra,Marca1,Marca2,Marca3,...,M2,M3,M4,M5,Conteo,Conteo1,Gerencia2,SubCanal2,Categoria,Nevera
0,1,0.000000,0.0,0.0,0.078244,0.000000,0.130406,0.000000,0.0,0.0,...,0.0,0.0,1.0,0.0,12,5,1,1,1,0
1,2,0.000000,0.0,0.0,0.117366,0.000000,0.888634,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,32,14,1,1,1,0
2,3,0.000000,0.0,0.0,0.000000,0.000000,0.086938,0.000000,0.0,0.0,...,0.0,0.0,1.0,0.0,11,3,1,1,1,0
3,4,0.101164,0.0,0.0,0.136927,0.000000,0.918509,632.820727,0.0,0.0,...,0.0,0.0,0.0,0.0,47,20,1,1,1,1
4,5,0.050582,0.0,0.0,0.000000,0.000000,3.578361,316.410336,0.0,0.0,...,0.0,0.0,0.0,0.0,65,27,1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13261,20414,0.000000,0.0,0.0,0.019561,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,1.0,0.0,6,4,1,3,1,0
13262,20462,0.000000,0.0,0.0,0.097805,0.039122,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,2,2,8,1,2,0
13263,20529,0.000000,0.0,0.0,0.880243,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,4,4,9,4,3,1
13264,20567,0.000000,0.0,0.0,0.019561,0.019561,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,2,2,10,26,1,0


In [592]:
Datoslogit2.head(3)

,Cliente,Marca1,Marca2,Marca3,Marca4,Marca5,Otra,Marca1,Marca2,Marca3,...,CapacidadEnvase_10,CapacidadEnvase_12,M1,M2,M3,M4,M5,Conteo,Conteo1,Conditional
0,1,0.0,0.0,0.0,0.078244,0.0,0.130406,0.0,0.0,0.0,...,3,0,0.0,0.0,0.0,1.0,0.0,12,5,2.0
1,2,0.0,0.0,0.0,0.117366,0.0,0.888634,0.0,0.0,0.0,...,4,1,1.0,0.0,0.0,0.0,0.0,32,14,2.0
2,3,0.0,0.0,0.0,0.000000,0.0,0.086938,0.0,0.0,0.0,...,1,0,0.0,0.0,0.0,1.0,0.0,11,3,0.0


In [596]:
Ventas3

,Cliente,Conditional
0,1,2
1,2,2
2,4,1
3,5,1
4,6,1
...,...,...
13237,20529,2
13238,20567,2
13239,20569,1
13240,20577,2


In [841]:
Ventas7=pd.read_csv('./Input2_clientes_venta.csv', sep=';')
Ventas7=Ventas7[(Ventas7["Año"]==2020) & (Ventas7["Mes"]>=7) &   (Ventas7["Mes"]<=8)& (Ventas7["Marca2"]!="Marca_1") &   (Ventas7["Marca2"]!="Marca_4")]
Ventas7=pd.pivot_table(Ventas7, values=['nr'], index=['Cliente'], columns=['Mes'], aggfunc=np.sum, fill_value=0)
Ventas7.columns = Ventas7.columns.droplevel(0)
Ventas7 = Ventas7.reset_index().rename_axis(None, axis=1)
Ventas7.columns = ['Cliente','mes1',"mes2"]
Ventas7["Conditional2"]=np.log(Ventas7['mes2']+1)/np.log(Ventas7['mes1']+1)
Ventas7=Ventas7.replace([np.inf, -np.inf], 0)
Ventas7=Ventas7[["Cliente","Conditional2"]]
Ventas7

,Cliente,Conditional2
0,1,0.000000
1,2,1.241875
2,4,0.982311
3,5,0.922317
4,6,0.944418
...,...,...
13237,20529,1.119707
13238,20567,0.000000
13239,20569,0.000000
13240,20577,0.000000


In [864]:
Ventas9=pd.read_csv('./Input2_clientes_venta.csv', sep=';')
Ventas9=Ventas9[(Ventas9["Año"]==2020) & (Ventas9["Mes"]>=7) &   (Ventas9["Mes"]<=8)& ((Ventas9["Marca2"]=="Marca_1") |   (Ventas9["Marca2"]=="Marca_4"))]
Ventas9=pd.pivot_table(Ventas9, values=['nr'], index=['Cliente'], columns=['Mes'], aggfunc=np.sum, fill_value=0)
Ventas9.columns = Ventas9.columns.droplevel(0)
Ventas9 = Ventas9.reset_index().rename_axis(None, axis=1)
Ventas9.columns = ['Cliente','mes1',"mes2"]
Ventas9["Conditional3"]=np.log(Ventas9['mes2']+1)/np.log(Ventas9['mes1']+1)
Ventas9=Ventas9.replace([np.inf, -np.inf], 0)
Ventas9=Ventas9[["Cliente","Conditional3"]]
Ventas9

,Cliente,Conditional3
0,1,1.198197
1,2,1.056972
2,3,1.000000
3,4,1.005546
4,5,1.076234
...,...,...
12576,19284,0.000000
12577,19291,0.000000
12578,19298,0.000000
12579,19307,0.000000


In [865]:
Datoslogit=Datapreparada= pd.merge(Periodo1, Ventas, how='left',  on='Cliente')
Datoslogit=Datapreparada= pd.merge(Datoslogit, VentasR, how='left',  on='Cliente')
Datoslogit=Datapreparada= pd.merge(Datoslogit, VentasS, how='left',  on='Cliente')
Datoslogit=Datapreparada= pd.merge(Datoslogit, Ventas3, how='left',  on='Cliente')
Datoslogit=Datapreparada= pd.merge(Datoslogit, Clientes, how='left',  on='Cliente')
Datoslogit=Datapreparada= pd.merge(Datoslogit, Ventas5, how='left',  on='Cliente')
Datoslogit=Datapreparada= pd.merge(Datoslogit, Ventas7, how='left',  on='Cliente')
Datoslogit=Datapreparada= pd.merge(Datoslogit, Ventas9, how='left',  on='Cliente')
Datoslogit2=Datoslogit.fillna(0)
Datoslogit2

,Cliente,Marca1,Marca2,Marca3,Marca4,Marca5,Otra,Marca1,Marca2,Marca3,...,Conteo,Conteo1,Conditional,Gerencia2,SubCanal2,Categoria,Nevera,Conditional1,Conditional2,Conditional3
0,1,0.000000,0.0,0.0,0.078244,0.000000,0.130406,0.000000,0.0,0.0,...,12,5,2.0,1,1,1,0,2.0,0.000000,1.198197
1,2,0.000000,0.0,0.0,0.117366,0.000000,0.888634,0.000000,0.0,0.0,...,32,14,2.0,1,1,1,0,2.0,1.241875,1.056972
2,3,0.000000,0.0,0.0,0.000000,0.000000,0.086938,0.000000,0.0,0.0,...,11,3,0.0,1,1,1,0,2.0,0.000000,1.000000
3,4,0.101164,0.0,0.0,0.136927,0.000000,0.918509,632.820727,0.0,0.0,...,47,20,1.0,1,1,1,1,2.0,0.982311,1.005546
4,5,0.050582,0.0,0.0,0.000000,0.000000,3.578361,316.410336,0.0,0.0,...,65,27,1.0,1,1,2,1,2.0,0.922317,1.076234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13261,20414,0.000000,0.0,0.0,0.019561,0.000000,0.000000,0.000000,0.0,0.0,...,6,4,2.0,1,3,1,0,0.0,1.000615,0.000000
13262,20462,0.000000,0.0,0.0,0.097805,0.039122,0.000000,0.000000,0.0,0.0,...,2,2,2.0,8,1,2,0,0.0,0.000000,0.000000
13263,20529,0.000000,0.0,0.0,0.880243,0.000000,0.000000,0.000000,0.0,0.0,...,4,4,2.0,9,4,3,1,0.0,1.119707,0.000000
13264,20567,0.000000,0.0,0.0,0.019561,0.019561,0.000000,0.000000,0.0,0.0,...,2,2,2.0,10,26,1,0,0.0,0.000000,0.000000


In [621]:
Datoslogit2.columns

Index(['Cliente', 'Marca1', 'Marca2', 'Marca3', 'Marca4', 'Marca5', 'Otra',
       'Marca1', 'Marca2', 'Marca3', 'Marca4', 'Marca5', 'Otra', 'Volumen',
       'nr', 'Descuento', 'Obsequio', 'CupoP', 'Marca_20', 'Marca_16',
       'Marca_9', 'Marca_38', 'Marca_39', 'CapacidadEnvase_9',
       'CapacidadEnvase_10', 'CapacidadEnvase_12', 'M1', 'M2', 'M3', 'M4',
       'M5', 'Conteo', 'Conteo1', 'Conditional', 'Gerencia2', 'SubCanal2',
       'Categoria', 'Nevera'],
      dtype='object')

In [866]:
#modlogitM1="M1~"+"+".join(Periodo1.iloc[:,1:26].columns)
modlogitM1='M1~Conditional3+Conditional2+Conteo+Conteo1+Marca3+Marca4+Marca5+Otra+Marca3+Marca4+Marca5+Otra+Volumen+nr+Obsequio+CupoP+Marca_20+Marca_38+Marca_39+CapacidadEnvase_9+Gerencia2+Nevera'
lm2   = sfm.ols(formula = modlogitM1, data = Datoslogit2).fit()
print(lm2.summary())
#Gerencia2	SubCanal2	Categoria	Nevera

                            OLS Regression Results                            
Dep. Variable:                     M1   R-squared:                       0.245
Model:                            OLS   Adj. R-squared:                  0.244
Method:                 Least Squares   F-statistic:                     204.4
Date:                Sun, 29 Nov 2020   Prob (F-statistic):               0.00
Time:                        10:31:20   Log-Likelihood:                -2984.8
No. Observations:               13266   AIC:                             6014.
Df Residuals:                   13244   BIC:                             6179.
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.1043      0.00

In [868]:
#modlogitM2="M2~"+"+".join(Periodo1.iloc[:,1:26].columns)
modlogitM2='M2~Conditional3+Conditional+Conteo1+Descuento+CupoP+Marca_16+Marca_9+Categoria+Nevera+Categoria'
lm3   = sfm.ols(formula = modlogitM2, data = Datoslogit2).fit()
print(lm3.summary())
#+Gerencia2+SubCanal2+Categoria+Nevera

                            OLS Regression Results                            
Dep. Variable:                     M2   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     51.76
Date:                Sun, 29 Nov 2020   Prob (F-statistic):           5.70e-93
Time:                        10:32:00   Log-Likelihood:                 11082.
No. Observations:               13266   AIC:                        -2.214e+04
Df Residuals:                   13256   BIC:                        -2.207e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.0044      0.004     -1.129   

In [869]:
#modlogitM3="M3~"+"+".join(Periodo1.iloc[:,1:26].columns)
modlogitM3='M3~Conditional3+Conditional1+Conteo1+Conteo+Marca_16+Marca_9+CapacidadEnvase_12+Gerencia2+SubCanal2+Categoria+Nevera'
lm4   = sfm.ols(formula = modlogitM3, data = Datoslogit2).fit()
print(lm4.summary())
#+Gerencia2+SubCanal2+Categoria+Nevera

                            OLS Regression Results                            
Dep. Variable:                     M3   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.310
Method:                 Least Squares   F-statistic:                     542.2
Date:                Sun, 29 Nov 2020   Prob (F-statistic):               0.00
Time:                        10:32:11   Log-Likelihood:                -5151.8
No. Observations:               13266   AIC:                         1.033e+04
Df Residuals:                   13254   BIC:                         1.042e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.1130      0

In [870]:
#modlogitM4="M4~"+"+".join(Periodo1.iloc[:,1:26].columns)
modlogitM4='M4~-1+Conditional3+Conditional1+Conteo1+Conteo+Obsequio+Marca_20+Marca_9+Marca_38+Marca_39+CapacidadEnvase_10+CapacidadEnvase_12+Gerencia2+SubCanal2+Categoria'
lm5   = sfm.ols(formula = modlogitM4, data = Datoslogit2).fit()
print(lm5.summary())
#+Gerencia2+SubCanal2+Categoria+Nevera

                                 OLS Regression Results                                
Dep. Variable:                     M4   R-squared (uncentered):                   0.421
Model:                            OLS   Adj. R-squared (uncentered):              0.421
Method:                 Least Squares   F-statistic:                              742.3
Date:                Sun, 29 Nov 2020   Prob (F-statistic):                        0.00
Time:                        10:32:19   Log-Likelihood:                         -4175.4
No. Observations:               13266   AIC:                                      8377.
Df Residuals:                   13253   BIC:                                      8474.
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

In [877]:
#modlogitM5="M5~"+"+".join(Periodo1.iloc[:,1:26].columns)
modlogitM5='M5~-1+Conditional3+Conteo+Obsequio+CupoP+Marca_20+Marca_9+Marca_38+Marca_39+CapacidadEnvase_10'
lm6   = sfm.ols(formula = modlogitM5, data = Datoslogit).fit()


print(lm6.summary())

                                 OLS Regression Results                                
Dep. Variable:                     M5   R-squared (uncentered):                   0.332
Model:                            OLS   Adj. R-squared (uncentered):              0.332
Method:                 Least Squares   F-statistic:                              703.7
Date:                Sun, 29 Nov 2020   Prob (F-statistic):                        0.00
Time:                        10:32:55   Log-Likelihood:                         -1154.8
No. Observations:               11314   AIC:                                      2326.
Df Residuals:                   11306   BIC:                                      2384.
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

In [879]:
Ventas8=pd.read_csv('./Input2_clientes_venta.csv', sep=';')
Ventas8=Ventas8[(Ventas8["Año"]==2020) & (Ventas8["Mes"]>=8) &   (Ventas8["Mes"]<=9)& (Ventas8["Marca2"]!="Marca_1") &   (Ventas8["Marca2"]!="Marca_4")]
Ventas8=pd.pivot_table(Ventas8, values=['nr'], index=['Cliente'], columns=['Mes'], aggfunc=np.sum, fill_value=0)
Ventas8.columns = Ventas8.columns.droplevel(0)
Ventas8 = Ventas8.reset_index().rename_axis(None, axis=1)
Ventas8.columns = ['Cliente','mes1',"mes2"]
Ventas8["Conditional2"]=np.log(Ventas8['mes2']+1)/np.log(Ventas8['mes1']+1)
Ventas8=Ventas8.replace([np.inf, -np.inf], 0)
Ventas8=Ventas8[["Cliente","Conditional2"]]
Ventas10=pd.read_csv('./Input2_clientes_venta.csv', sep=';')
Ventas10=Ventas10[(Ventas10["Año"]==2020) & (Ventas10["Mes"]>=8) &   (Ventas10["Mes"]<=9)& ((Ventas10["Marca2"]=="Marca_1") |   (Ventas10["Marca2"]=="Marca_4"))]
Ventas10=pd.pivot_table(Ventas10, values=['nr'], index=['Cliente'], columns=['Mes'], aggfunc=np.sum, fill_value=0)
Ventas10.columns = Ventas10.columns.droplevel(0)
Ventas10 = Ventas10.reset_index().rename_axis(None, axis=1)
Ventas10.columns = ['Cliente','mes1',"mes2"]
Ventas10["Conditional3"]=np.log(Ventas10['mes2']+1)/np.log(Ventas10['mes1']+1)
Ventas10=Ventas10.replace([np.inf, -np.inf], 0)
Ventas10=Ventas10[["Cliente","Conditional3"]]
Ventas10

,Cliente,Conditional3
0,1,0.758633
1,2,0.985140
2,3,0.000000
3,4,0.857805
4,5,0.521557
...,...,...
13228,19871,0.000000
13229,19876,0.000000
13230,19885,0.000000
13231,19891,0.000000


In [880]:
DATAPrueba= pd.merge(Periodo2, VentasR, how='left',  on='Cliente')
DATAPrueba= pd.merge(DATAPrueba, VentasT, how='left',  on='Cliente')
DATAPrueba= pd.merge(DATAPrueba, Clientes, how='left',  on='Cliente')
DATAPrueba= pd.merge(DATAPrueba, Ventas4, how='left',  on='Cliente')
DATAPrueba= pd.merge(DATAPrueba, Ventas6, how='left',  on='Cliente')
DATAPrueba= pd.merge(DATAPrueba, Ventas8, how='left',  on='Cliente')
DATAPrueba= pd.merge(DATAPrueba, Ventas10, how='left',  on='Cliente')
DATAPrueba=DATAPrueba.fillna(0)
DATAPrueba

,Cliente,Marca1,Marca2,Marca3,Marca4,Marca5,Otra,Marca1,Marca2,Marca3,...,Conteo,Conteo1,Gerencia2,SubCanal2,Categoria,Nevera,Conditional,Conditional1,Conditional2,Conditional3
0,1,0.000000,0.0,0.0,0.078244,0.000000,0.021734,0.000000,0.0,0.0,...,12,7,1,1,1,0,2.0,1.0,0.995975,0.758633
1,2,0.067443,0.0,0.0,0.000000,0.000000,0.715786,421.880521,0.0,0.0,...,32,16,1,1,1,0,1.0,1.0,0.878290,0.985140
2,3,0.000000,0.0,0.0,0.117366,0.000000,0.000000,0.000000,0.0,0.0,...,11,4,1,1,1,0,2.0,1.0,0.000000,0.000000
3,4,0.000000,0.0,0.0,0.000000,0.000000,0.217344,0.000000,0.0,0.0,...,47,18,1,1,1,1,1.0,1.0,0.722169,0.857805
4,5,0.000000,0.0,0.0,0.000000,0.000000,0.036334,0.000000,0.0,0.0,...,65,16,1,1,2,1,1.0,1.0,0.737607,0.521557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13659,20411,0.000000,0.0,0.0,0.352097,0.156488,0.000000,0.000000,0.0,0.0,...,6,6,10,31,5,0,2.0,0.0,1.145982,0.000000
13660,20414,0.000000,0.0,0.0,0.039122,0.000000,0.000000,0.000000,0.0,0.0,...,6,6,1,3,1,0,2.0,0.0,1.153926,0.000000
13661,20434,0.000000,0.0,0.0,0.019561,0.000000,0.000000,0.000000,0.0,0.0,...,2,2,10,31,3,0,2.0,0.0,0.000000,0.000000
13662,20456,0.000000,0.0,0.0,0.009780,0.009780,0.000000,0.000000,0.0,0.0,...,2,2,10,1,2,0,2.0,0.0,0.000000,0.000000


In [881]:
ynew1=lm2.predict(DATAPrueba.iloc[:,1:36])
ynew2=lm3.predict(DATAPrueba.iloc[:,1:36])
ynew3=lm4.predict(DATAPrueba.iloc[:,1:36])
ynew4=lm5.predict(DATAPrueba.iloc[:,1:36])
ynew5=lm6.predict(DATAPrueba.iloc[:,1:36])
ynew1=1/(1+np.exp(-ynew1))
ynew2=1/(1+np.exp(-ynew2))
ynew3=1/(1+np.exp(-ynew3))
ynew4=1/(1+np.exp(-ynew4))
ynew5=1/(1+np.exp(-ynew5))
ynew1=pd.DataFrame(ynew1)
ynew2=pd.DataFrame(ynew2)
ynew3=pd.DataFrame(ynew3)
ynew4=pd.DataFrame(ynew4)
ynew5=pd.DataFrame(ynew5)
Salida=pd.concat([Periodo2.iloc[:, 0],ynew1, ynew2,ynew3,ynew4,ynew5], axis=1, sort=False)
Salida

,Cliente,0,0,0,0,0
0,1,0.472504,0.499170,0.482171,0.576667,0.516050
1,2,0.546924,0.500233,0.504696,0.502738,0.504798
2,3,0.480306,0.499052,0.487398,0.582738,0.519148
3,4,0.499103,0.499720,0.511158,0.504428,0.509267
4,5,0.509130,0.499584,0.536042,0.512446,0.511330
...,...,...,...,...,...,...
13659,20411,0.512543,0.500878,0.525798,0.669784,0.609784
13660,20414,0.470695,0.498807,0.479591,0.585208,0.517992
13661,20434,0.505131,0.500334,0.518031,0.621485,0.517067
13662,20456,0.518079,0.499816,0.499579,0.646265,0.608902


In [883]:
Salida.to_csv('out.csv')

In [310]:
# machine learning: XGB
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from xgboost.sklearn import XGBRegressor # wrapper

In [443]:
T1=Datoslogit.iloc[:,0:26]
Tr1=Datoslogit["M1"]
Tr2=Datoslogit["M2"]
Tr3=Datoslogit["M3"]
Tr4=Datoslogit["M4"]
Tr5=Datoslogit["M5"]
T1.columns = ['Cliente','var1','var2', 'var3', 'var4', 'var5', 'var6', 'var7', 'var8', 'var9', 'var10', 'var11', 'var12', 'var13', 'var14', 'var15', 'var16', 'var17', 'var18', 'var19', 'var20', 'var21', 'var22', 'var23', 'var24', 'var25']

In [535]:
T1=Datoslogit2
T1.columns = ['Cliente','var1','var2', 'var3', 'var4', 'var5', 'var6', 'var7', 'var8',
              'var9', 'var10', 'var11', 'var12', 'var13', 'var14', 'var15', 'var16', 'var17',
              'var18', 'var19', 'var20', 'var21', 'var22', 'var23', 'var24', 'var25',
             'M1','M2','M3','M4','M5','var26','var27','var28','var29','var30','var31']

list3 = ['var1','var2', 'var3', 'var4', 'var5', 'var6', 'var7', 'var8',
              'var9', 'var10', 'var11', 'var12', 'var13', 'var14', 'var15', 'var16', 'var17',
              'var18', 'var19', 'var20', 'var21', 'var22', 'var23', 'var24', 'var25',
                'var26','var27','var28','var29','var30','var31']
T1=T1[list3]
T1

,var1,var2,var3,var4,var5,var6,var7,var8,var9,var10,...,var22,var23,var24,var25,var26,var27,var28,var29,var30,var31
0,0.000000,0.0,0.0,0.078244,0.000000,0.130406,0.000000,0.0,0.0,488.267995,...,1,0,3,0,12,5,1,1,1,0
1,0.000000,0.0,0.0,0.117366,0.000000,0.888634,0.000000,0.0,0.0,719.720948,...,1,0,4,1,32,14,1,1,1,0
2,0.000000,0.0,0.0,0.000000,0.000000,0.086938,0.000000,0.0,0.0,0.000000,...,0,0,1,0,11,3,1,1,1,0
3,0.101164,0.0,0.0,0.136927,0.000000,0.918509,632.820727,0.0,0.0,815.028471,...,1,1,5,0,47,20,1,1,1,1
4,0.050582,0.0,0.0,0.000000,0.000000,3.578361,316.410336,0.0,0.0,0.000000,...,0,1,2,0,65,27,1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13261,0.000000,0.0,0.0,0.019561,0.000000,0.000000,0.000000,0.0,0.0,114.742688,...,1,0,2,0,6,4,1,3,1,0
13262,0.000000,0.0,0.0,0.097805,0.039122,0.000000,0.000000,0.0,0.0,593.298287,...,1,0,2,0,2,2,8,1,2,0
13263,0.000000,0.0,0.0,0.880243,0.000000,0.000000,0.000000,0.0,0.0,5408.754393,...,1,0,2,0,4,4,9,4,3,1
13264,0.000000,0.0,0.0,0.019561,0.019561,0.000000,0.000000,0.0,0.0,121.146074,...,1,0,2,0,2,2,10,26,1,0


In [558]:
# base parameters
params = {
    'booster': 'gbtree', 
    'objective': 'reg:logistic', # regression task
    'subsample': 0.8, # 80% of data to grow trees and prevent overfitting
    'colsample_bytree': 0.85, # 85% of features used
    'eta': 0.1, 
    'max_depth': 10, 
    'seed': 42} # for reproducible results
# evaluation metric: rmspe
# Root Mean Square Percentage Error
# code chunk shared at Kaggle

def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat / y-1) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y, yhat)

In [541]:
T2=Periodo2.iloc[:,1:33]
T2.columns = ['var1','var2', 'var3', 'var4', 'var5', 'var6', 'var7', 'var8',
              'var9', 'var10', 'var11', 'var12', 'var13', 'var14', 'var15', 'var16', 'var17',
              'var18', 'var19', 'var20', 'var21', 'var22', 'var23', 'var24', 'var25',
                'var26','var27','var28','var29','var30','var31']
T2
T2= xgb.DMatrix(T2)
#Periodo2

In [538]:
print(X_train.shape)
print(T2.shape)
print(Periodo2.shape)
print(T1.shape)

(9286, 31)
(13664, 31)
(13664, 32)
(13266, 31)


In [559]:
X_train, X_test, y_train, y_test = train_test_split(T1, Tr1, 
                                                    test_size = 0.3, # 30% for the evaluation set
                                                    random_state = 42)
# XGB with xgboost library
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

watchlist = [(dtrain, 'train'), (dtest, 'test')]

xgb_model = xgb.train(params, dtrain, 500, evals = watchlist,
                      early_stopping_rounds = 100, feval = rmspe_xg, verbose_eval = True)

ynew1 = xgb_model.predict(T2)
ynew1

[0]	train-rmse:0.46569	test-rmse:0.46905	train-rmspe:nan	test-rmspe:nan
Multiple eval metrics have been passed: 'test-rmspe' will be used for early stopping.

Will train until test-rmspe hasn't improved in 100 rounds.
[1]	train-rmse:0.43663	test-rmse:0.44330	train-rmspe:inf	test-rmspe:inf
[2]	train-rmse:0.41018	test-rmse:0.42079	train-rmspe:inf	test-rmspe:inf
[3]	train-rmse:0.38700	test-rmse:0.40125	train-rmspe:inf	test-rmspe:inf


C:\Users\camil\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\Users\camil\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


[4]	train-rmse:0.36665	test-rmse:0.38469	train-rmspe:inf	test-rmspe:inf
[5]	train-rmse:0.34861	test-rmse:0.37085	train-rmspe:inf	test-rmspe:inf
[6]	train-rmse:0.33272	test-rmse:0.35906	train-rmspe:inf	test-rmspe:inf
[7]	train-rmse:0.31878	test-rmse:0.34926	train-rmspe:inf	test-rmspe:inf
[8]	train-rmse:0.30620	test-rmse:0.34115	train-rmspe:inf	test-rmspe:inf
[9]	train-rmse:0.29544	test-rmse:0.33406	train-rmspe:inf	test-rmspe:inf
[10]	train-rmse:0.28603	test-rmse:0.32868	train-rmspe:inf	test-rmspe:inf
[11]	train-rmse:0.27787	test-rmse:0.32415	train-rmspe:inf	test-rmspe:inf
[12]	train-rmse:0.27033	test-rmse:0.32043	train-rmspe:inf	test-rmspe:inf
[13]	train-rmse:0.26432	test-rmse:0.31740	train-rmspe:inf	test-rmspe:inf
[14]	train-rmse:0.25810	test-rmse:0.31499	train-rmspe:inf	test-rmspe:inf
[15]	train-rmse:0.25245	test-rmse:0.31278	train-rmspe:inf	test-rmspe:inf
[16]	train-rmse:0.24774	test-rmse:0.31082	train-rmspe:inf	test-rmspe:inf
[17]	train-rmse:0.24265	test-rmse:0.30902	train-rmspe:inf

array([0.00314209, 0.06421369, 0.00219179, ..., 0.00244467, 0.00416375,
       0.00582615], dtype=float32)

In [553]:
X_train, X_test, y_train, y_test = train_test_split(T1, Tr2, 
                                                    test_size = 0.3, # 30% for the evaluation set
                                                    random_state = 42)
# XGB with xgboost library
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

watchlist = [(dtrain, 'train'), (dtest, 'test')]

xgb_model = xgb.train(params, dtrain, 500, evals = watchlist,
                      early_stopping_rounds = 100, feval = rmspe_xg, verbose_eval = True)
ynew2 = xgb_model.predict(T2)
ynew2

[0]	train-error:0.01034	test-error:0.01307	train-rmspe:nan	test-rmspe:nan
Multiple eval metrics have been passed: 'test-rmspe' will be used for early stopping.

Will train until test-rmspe hasn't improved in 100 rounds.
[1]	train-error:0.00980	test-error:0.01307	train-rmspe:inf	test-rmspe:inf
[2]	train-error:0.00969	test-error:0.01256	train-rmspe:inf	test-rmspe:inf
[3]	train-error:0.00958	test-error:0.01256	train-rmspe:inf	test-rmspe:inf
[4]	train-error:0.00980	test-error:0.01256	train-rmspe:inf	test-rmspe:inf
[5]	train-error:0.00980	test-error:0.01281	train-rmspe:inf	test-rmspe:inf
[6]	train-error:0.00969	test-error:0.01256	train-rmspe:inf	test-rmspe:inf


C:\Users\camil\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\Users\camil\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


[7]	train-error:0.00991	test-error:0.01256	train-rmspe:inf	test-rmspe:inf
[8]	train-error:0.00937	test-error:0.01281	train-rmspe:inf	test-rmspe:inf
[9]	train-error:0.00958	test-error:0.01332	train-rmspe:inf	test-rmspe:inf
[10]	train-error:0.00958	test-error:0.01307	train-rmspe:inf	test-rmspe:inf
[11]	train-error:0.00937	test-error:0.01231	train-rmspe:inf	test-rmspe:inf
[12]	train-error:0.00948	test-error:0.01231	train-rmspe:inf	test-rmspe:inf
[13]	train-error:0.00958	test-error:0.01256	train-rmspe:inf	test-rmspe:inf
[14]	train-error:0.00937	test-error:0.01256	train-rmspe:inf	test-rmspe:inf
[15]	train-error:0.00948	test-error:0.01256	train-rmspe:inf	test-rmspe:inf
[16]	train-error:0.00926	test-error:0.01256	train-rmspe:inf	test-rmspe:inf
[17]	train-error:0.00926	test-error:0.01231	train-rmspe:inf	test-rmspe:inf
[18]	train-error:0.00915	test-error:0.01231	train-rmspe:inf	test-rmspe:inf
[19]	train-error:0.00905	test-error:0.01231	train-rmspe:inf	test-rmspe:inf
[20]	train-error:0.00894	tes

array([0.00038845, 0.00090022, 0.00021969, ..., 0.00078276, 0.00025816,
       0.00034154], dtype=float32)

In [554]:
X_train, X_test, y_train, y_test = train_test_split(T1, Tr3, 
                                                    test_size = 0.3, # 30% for the evaluation set
                                                    random_state = 42)
# XGB with xgboost library
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

watchlist = [(dtrain, 'train'), (dtest, 'test')]

xgb_model = xgb.train(params, dtrain, 500, evals = watchlist,
                      early_stopping_rounds = 100, feval = rmspe_xg, verbose_eval = True)
ynew3 = xgb_model.predict(T2)
ynew3

[0]	train-error:0.13407	test-error:0.20151	train-rmspe:nan	test-rmspe:nan
Multiple eval metrics have been passed: 'test-rmspe' will be used for early stopping.

Will train until test-rmspe hasn't improved in 100 rounds.
[1]	train-error:0.12094	test-error:0.19347	train-rmspe:nan	test-rmspe:nan
[2]	train-error:0.11781	test-error:0.18367	train-rmspe:inf	test-rmspe:inf
[3]	train-error:0.11210	test-error:0.17889	train-rmspe:inf	test-rmspe:inf
[4]	train-error:0.10909	test-error:0.18166	train-rmspe:inf	test-rmspe:inf


C:\Users\camil\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\Users\camil\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


[5]	train-error:0.10694	test-error:0.17789	train-rmspe:inf	test-rmspe:inf
[6]	train-error:0.10457	test-error:0.17789	train-rmspe:inf	test-rmspe:inf
[7]	train-error:0.10220	test-error:0.17663	train-rmspe:inf	test-rmspe:inf
[8]	train-error:0.09918	test-error:0.17437	train-rmspe:inf	test-rmspe:inf
[9]	train-error:0.09735	test-error:0.17513	train-rmspe:inf	test-rmspe:inf
[10]	train-error:0.09660	test-error:0.17412	train-rmspe:inf	test-rmspe:inf
[11]	train-error:0.09369	test-error:0.17261	train-rmspe:inf	test-rmspe:inf
[12]	train-error:0.09304	test-error:0.17261	train-rmspe:inf	test-rmspe:inf
[13]	train-error:0.09251	test-error:0.17186	train-rmspe:inf	test-rmspe:inf
[14]	train-error:0.09121	test-error:0.17286	train-rmspe:inf	test-rmspe:inf
[15]	train-error:0.09067	test-error:0.17136	train-rmspe:inf	test-rmspe:inf
[16]	train-error:0.08949	test-error:0.17337	train-rmspe:inf	test-rmspe:inf
[17]	train-error:0.08701	test-error:0.17487	train-rmspe:inf	test-rmspe:inf
[18]	train-error:0.08432	test-

array([0.01750663, 0.02071619, 0.04897742, ..., 0.00088613, 0.00085239,
       0.00102825], dtype=float32)

In [555]:
X_train, X_test, y_train, y_test = train_test_split(T1, Tr4, 
                                                    test_size = 0.3, # 30% for the evaluation set
                                                    random_state = 42)
# XGB with xgboost library
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

watchlist = [(dtrain, 'train'), (dtest, 'test')]

xgb_model = xgb.train(params, dtrain, 500, evals = watchlist,
                      early_stopping_rounds = 100, feval = rmspe_xg, verbose_eval = True)
ynew4 = xgb_model.predict(T2)
ynew4

[0]	train-error:0.10220	test-error:0.14648	train-rmspe:nan	test-rmspe:nan
Multiple eval metrics have been passed: 'test-rmspe' will be used for early stopping.

Will train until test-rmspe hasn't improved in 100 rounds.
[1]	train-error:0.09584	test-error:0.13995	train-rmspe:inf	test-rmspe:inf
[2]	train-error:0.09272	test-error:0.13844	train-rmspe:inf	test-rmspe:inf
[3]	train-error:0.09326	test-error:0.13518	train-rmspe:inf	test-rmspe:inf
[4]	train-error:0.09186	test-error:0.13317	train-rmspe:inf	test-rmspe:inf


C:\Users\camil\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\Users\camil\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


[5]	train-error:0.08981	test-error:0.13492	train-rmspe:inf	test-rmspe:inf
[6]	train-error:0.08906	test-error:0.13492	train-rmspe:inf	test-rmspe:inf
[7]	train-error:0.08906	test-error:0.13417	train-rmspe:inf	test-rmspe:inf
[8]	train-error:0.08755	test-error:0.13518	train-rmspe:inf	test-rmspe:inf
[9]	train-error:0.08690	test-error:0.13769	train-rmspe:inf	test-rmspe:inf
[10]	train-error:0.08561	test-error:0.13719	train-rmspe:inf	test-rmspe:inf
[11]	train-error:0.08432	test-error:0.13518	train-rmspe:inf	test-rmspe:inf
[12]	train-error:0.08324	test-error:0.13442	train-rmspe:inf	test-rmspe:inf
[13]	train-error:0.08227	test-error:0.13317	train-rmspe:inf	test-rmspe:inf
[14]	train-error:0.08012	test-error:0.13442	train-rmspe:inf	test-rmspe:inf
[15]	train-error:0.07808	test-error:0.13417	train-rmspe:inf	test-rmspe:inf
[16]	train-error:0.07732	test-error:0.13392	train-rmspe:inf	test-rmspe:inf
[17]	train-error:0.07614	test-error:0.13342	train-rmspe:inf	test-rmspe:inf
[18]	train-error:0.07506	test-

array([0.410197  , 0.03459727, 0.1894802 , ..., 0.00429689, 0.02093794,
       0.05320935], dtype=float32)

In [556]:
X_train, X_test, y_train, y_test = train_test_split(T1, Tr5, 
                                                    test_size = 0.3, # 30% for the evaluation set
                                                    random_state = 42)
# XGB with xgboost library
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

watchlist = [(dtrain, 'train'), (dtest, 'test')]

xgb_model = xgb.train(params, dtrain, 500, evals = watchlist,
                      early_stopping_rounds = 100, feval = rmspe_xg, verbose_eval = True)
ynew5 = xgb_model.predict(T2)
ynew5

[0]	train-error:0.07247	test-error:0.10578	train-rmspe:nan	test-rmspe:nan
Multiple eval metrics have been passed: 'test-rmspe' will be used for early stopping.

Will train until test-rmspe hasn't improved in 100 rounds.
[1]	train-error:0.06698	test-error:0.09648	train-rmspe:nan	test-rmspe:nan
[2]	train-error:0.06311	test-error:0.09121	train-rmspe:inf	test-rmspe:inf
[3]	train-error:0.05998	test-error:0.09095	train-rmspe:inf	test-rmspe:inf
[4]	train-error:0.05977	test-error:0.08894	train-rmspe:inf	test-rmspe:inf


C:\Users\camil\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\Users\camil\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


[5]	train-error:0.05804	test-error:0.08945	train-rmspe:inf	test-rmspe:inf
[6]	train-error:0.05686	test-error:0.09271	train-rmspe:inf	test-rmspe:inf
[7]	train-error:0.05460	test-error:0.09095	train-rmspe:inf	test-rmspe:inf
[8]	train-error:0.05514	test-error:0.09070	train-rmspe:inf	test-rmspe:inf
[9]	train-error:0.05460	test-error:0.09045	train-rmspe:inf	test-rmspe:inf
[10]	train-error:0.05266	test-error:0.09221	train-rmspe:inf	test-rmspe:inf
[11]	train-error:0.05137	test-error:0.08945	train-rmspe:inf	test-rmspe:inf
[12]	train-error:0.05040	test-error:0.08970	train-rmspe:inf	test-rmspe:inf
[13]	train-error:0.04932	test-error:0.09070	train-rmspe:inf	test-rmspe:inf
[14]	train-error:0.04921	test-error:0.08920	train-rmspe:inf	test-rmspe:inf
[15]	train-error:0.04749	test-error:0.08769	train-rmspe:inf	test-rmspe:inf
[16]	train-error:0.04663	test-error:0.08844	train-rmspe:inf	test-rmspe:inf
[17]	train-error:0.04631	test-error:0.08769	train-rmspe:inf	test-rmspe:inf
[18]	train-error:0.04512	test-

array([0.03458392, 0.01061505, 0.01737054, ..., 0.00371515, 0.02745661,
       0.08541629], dtype=float32)

In [560]:
#ynew1=1/(1+np.exp(-ynew1))
#ynew2=1/(1+np.exp(-ynew2))
#ynew3=1/(1+np.exp(-ynew3))
#ynew4=1/(1+np.exp(-ynew4))
#ynew5=1/(1+np.exp(-ynew5))

ynew1=pd.DataFrame(ynew1)
ynew2=pd.DataFrame(ynew2)
ynew3=pd.DataFrame(ynew3)
ynew4=pd.DataFrame(ynew4)
ynew5=pd.DataFrame(ynew5)
Salida=pd.concat([Periodo2.iloc[:, 0],ynew1, ynew2,ynew3,ynew4,ynew5], axis=1, sort=False)
Salida

,Cliente,0,0,0,0,0
0,1,0.003142,0.000388,0.017507,0.410197,0.034584
1,2,0.064214,0.000900,0.020716,0.034597,0.010615
2,3,0.002192,0.000220,0.048977,0.189480,0.017371
3,4,0.009005,0.000473,0.031356,0.058325,0.017231
4,5,0.019662,0.000553,0.022610,0.123795,0.012451
...,...,...,...,...,...,...
13659,20411,0.004200,0.001011,0.001272,0.947526,0.981003
13660,20414,0.001066,0.000213,0.002189,0.425491,0.013155
13661,20434,0.002445,0.000783,0.000886,0.004297,0.003715
13662,20456,0.004164,0.000258,0.000852,0.020938,0.027457


In [550]:
Salida.to_csv('out1.csv') 

In [248]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, GlobalMaxPooling1D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
try:
    tf.set_random_seed(1337)                    # set the random seed for reproducibility
except:
    tf.random.set_seed(1337)                     # NOTE: Newer version of tensorflow uses tf.random.set_seed
np.random.seed(1337)

In [325]:
model = Sequential()
model.add(Embedding(3000, 32, input_length=25))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
#model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.summary()
model.fit(T1, Tr1, validation_split=0.25, epochs=2)

Epoch 1/2


InvalidArgumentError:  indices[0,12] = 7971 is not in [0, 3000)
	 [[node sequential_35/embedding_31/embedding_lookup (defined at <ipython-input-325-7d47183b7727>:10) ]] [Op:__inference_train_function_36206]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_35/embedding_31/embedding_lookup:
 sequential_35/embedding_31/embedding_lookup/35828 (defined at C:\Users\camil\Anaconda3\lib\contextlib.py:112)

Function call stack:
train_function


In [304]:
model = Sequential()
model.add(Dense(8, input_dim=8, kernel_initializer='normal', activation='relu'))
model.add(Dense(2670, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.summary()
model.fit(Datoslogit.iloc[:,1:26], Datoslogit.iloc[:,0], validation_split=0.25, epochs=2)

Epoch 1/2


ValueError: in user code:

    C:\Users\camil\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\camil\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\camil\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\camil\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\camil\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\camil\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
 

In [291]:
Datoslogit["M1"]


0        0.0
1        1.0
2        0.0
3        0.0
4        0.0
        ... 
13261    0.0
13262    0.0
13263    0.0
13264    0.0
13265    0.0
Name: M1, Length: 13266, dtype: float64